※ここに掲載されているコードには誤りがあります  
　結果の統合のところで総和が１になるように正規化されていない

# 11/10ゼミ

**提案手法 MMR-E**  
**Phase 1**  
区間重要度の中心を各手法で推定  
* $k$ 番目を除いた $n-1$ 個の $w_i$ に対して EV, GM, ALD で $w_{i}^{C}$ を求める（この時点でそれらの総和は１となる）
* 上記で求めた $w_{i}^{C}$ の和を $\mu$ 倍することで、 $k$ 番目の区間重要度中心は $1-\mu$ となる

**Phase 2**  
各 $k$ に対して、以下の線形最適化問題を解き、最適値を $d_{\overline{k}}^{*}$ とする：  
$$
\begin{align*}
\operatorname{minimize} ~~ & ~~
d_{\overline{k}} = \sum_{j \neq k} l_j, \\
\text{subject to} ~~ & ~~
a_{ij} \left( \mu w_{j}^{C} - l_j \right) \leq \mu w_{i}^{C} + l_i, &&
i \neq j, k, ~~ & j \neq k \\ & ~~
a_{kj} \left( \mu w_{j}^{C} - l_j \right) \leq 1-\mu + l_k, &&
j \neq k \\ & ~~
a_{ik} \{ \left(1- \mu \right) - l_k \} \leq \mu w_{i}^{C} + l_i, &&
i \neq k \\ & ~~
\left( 1 - \mu \right) + l_k + \sum_{i \neq j, k} \left( \mu w_{i}^{C} + l_i \right) + \mu w_{j}^{C} - l_j \geq 1, &&
i \in N \backslash k \\ & ~~
\left( 1 - \mu \right) - l_k + \sum_{i \neq j, k} \left( \mu w_{i}^{C} - l_i \right) + \mu w_{j}^{C} + l_j \leq 1, &&
i \in N \backslash k \\ & ~~
\mu w_{i}^{C} - l_i \geq \epsilon, &&
i \in N, ~~ & i \neq k \\ & ~~

\left( 1 - \mu \right) - l_k \geq \epsilon \\ & ~~
\sum_{i \neq k} \left( \mu w_{i}^{C} + l_i \right) + \left( 1 - \mu \right) - l_k \geq 1 \\ & ~~
\sum_{i \neq k} \left( \mu w_{i}^{C} - l_i \right) + \left( 1 - \mu \right) + l_k \leq 1 \\ & ~~


\end{align*}
$$



**Phase 3**  
各 $k$ に対して、以下の線形最適化問題を解き、最適値を $l_{k}^{*}$ とする：  
$$
\begin{align*}
\operatorname{minimize} ~~ & ~~
l_k, \\
\text{subject to} ~~ & ~~
a_{ij} \left( \mu w_{j}^{C} - l_j \right) \leq \mu w_{i}^{C} + l_i, &&
i \neq j, k, ~~ & j \neq k \\ & ~~
a_{kj} \left( \mu w_{j}^{C} - l_j \right) \leq 1-\mu + l_k, &&
j \neq k \\ & ~~
a_{ik} \{ \left(1- \mu \right) - l_k \} \leq \mu w_{i}^{C} + l_i, &&
i \neq k \\ & ~~
\left( 1 - \mu \right) + l_k + \sum_{i \neq j, k} \left( \mu w_{i}^{C} + l_i \right) + \mu w_{j}^{C} - l_j \geq 1, &&
i \in N \backslash k \\ & ~~
\left( 1 - \mu \right) - l_k + \sum_{i \neq j, k} \left( \mu w_{i}^{C} - l_i \right) + \mu w_{j}^{C} + l_j \leq 1, &&
i \in N \backslash k \\ & ~~
\mu w_{i}^{C} - l_i \geq \epsilon, &&
i \in N, ~~ & i \neq k \\ & ~~

\left( 1 - \mu \right) - l_k \geq \epsilon \\ & ~~
\sum_{i \neq k} \left( \mu w_{i}^{C} + l_i \right) + \left( 1 - \mu \right) - l_k \geq 1 \\ & ~~
\sum_{i \neq k} \left( \mu w_{i}^{C} - l_i \right) + \left( 1 - \mu \right) + l_k \leq 1 \\ & ~~
\sum_{i \neq j, k} l_k = d_{\overline{k}}^{*}\\ & ~~


\end{align*}
$$

In [12]:
using LaTeXStrings

In [13]:
include("./libs/display-latex.jl")
include("./libs/crisp-pcm.jl")
include("./libs/solve-deterministic-ahp.jl")
include("./libs/MMR.jl")

MMR (generic function with 1 method)

In [14]:
# 完全整合
# A = [
#     1 1 1 1
#     1 1 1 1
#     1 1 1 1
#     1 1 1 1.0
# ]
# A = [
#     1   3   5   7
#     1/3 1   5/3 7/3
#     1/5 3/5 1   7/5
#     1/7 3/7 5/7 1
# ]
# A = [
#     1 1 1/3
#     1 1 1/3
#     3 3 1
# ]

# CI = 0.01
# A = [
#     1 2 4/3 4/3
#     1/2 1 1 1
#     3/4 1 1 3/2
#     3/4 1 2/3 1.0
# ]
# A = [
#     1 1 2/3 1
#     1 1 4/9 2/3
#     3/2 9/4 1 1
#     1 3/2 1 1.0
# ]

# CI = 0.15
# A = [
#     1 1 2 2
#     1 1 3 1
#     1/2 1/3 1 3
#     1/2 1 1/3 1
# ]

# エラーケース EVでダメなやつ
# A = [
#     1.0  1.0  2.0  3.0  3.0
#     1.0  1.0  2.0  3.0  5.0
#     1/2  1/2  1.0  1.0  3.0
#     1/3  1/3  1.0  1.0  2.0
#     1/3  1/5  1/3  1/2  1.0
# ]

# エラーケース GMでダメなやつ
# A = [
#     1.0  1.0  2.0  3.0  8.0
#     1.0  1.0  2.0  2.0  3.0
#     1/2  1/2  1.0  2.0  6.0
#     1/3  1/2  1/2  1.0  5.0
#     1/8  1/3  1/6  1/5  1.0
# ]

# エラーケース ADでダメなやつ
# A = [
#     1.0  2.0   2.0   3.0   2.0
#     1/2  1.0   1.0   2.0   4.0
#     1/2  1.0   1.0   1.0   4.0
#     1/3  1/2   1.0   1.0   4.0
#     1/2  1/4  1/4  1/4  1.0
# ]

# エラーケース ADでダメなやつ
# 分数表記はOK
# A = [
#     1.0 1.0 2.0 3.0 6.0
#     1.0 1.0 2.0 2.0 6.0
#     1/2 1/2 1.0 1.0 5.0
#     1/3 1/2 1.0 1.0 1.0
#     1/6 1/6 1/5 1.0 1.0
# ]

# 実数表記はNG
A = [
    1.000000 1.000000 2.000000 3.000000 6.000000
    1.000000 1.000000 2.000000 2.000000 6.000000
    0.500000 0.500000 1.000000 1.000000 5.000000
    0.333333 0.500000 1.000000 1.000000 1.000000
    0.166667 0.166667 0.200000 1.000000 1.000000
]

display(L"A = %$(matrixLaTeXString(A))")

L"$A = \begin{pmatrix} 1.0 & 1.0 & 2.0 & 3.0 & 6.0 \\1.0 & 1.0 & 2.0 & 2.0 & 6.0 \\0.5 & 0.5 & 1.0 & 1.0 & 5.0 \\0.333 & 0.5 & 1.0 & 1.0 & 1.0 \\0.167 & 0.167 & 0.2 & 1.0 & 1.0 \\ \end{pmatrix}$"

In [15]:
consistencyIndex(A)

0.053778261838655084

In [16]:
EV₁ = EV(A)
GM₁ = GM(A)
ALD₁ = ALD(A)
display(L"EV₁ = %$(VectorLaTeXString(EV₁))")
display(L"GM₁ = %$(VectorLaTeXString(GM₁))")
display(L"ALD₁ = %$(VectorLaTeXString(ALD₁))")

L"$EV₁ = \begin{pmatrix} 0.331 \\ 0.308 \\ 0.178 \\ 0.12 \\ 0.062 \end{pmatrix}$"

L"$GM₁ = \begin{pmatrix} 0.339 \\ 0.313 \\ 0.173 \\ 0.116 \\ 0.059 \end{pmatrix}$"

L"$ALD₁ = \begin{pmatrix} 0.333 \\ 0.333 \\ 0.167 \\ 0.111 \\ 0.056 \end{pmatrix}$"

In [17]:
# D = phase2(A, EV)
# display(L"D = %$(VectorLaTeXString(D))")

In [18]:
# D = phase2(A, AD)
# display(L"D = %$(VectorLaTeXString(D))")

In [19]:
# D = phase2(A, GM)
# display(L"D = %$(VectorLaTeXString(D))")

In [20]:
# solution = MMR(A, EV)
# W = solution.W
# # w̅̅ᴸ = solution.w̅̅ᴸ
# # w̅̅ᵁ = solution.w̅̅ᵁ
# display(L"EV = %$(intervalVectorLaTeXString(W))")

In [21]:
solution = MMR(A, ALD)
W = solution.W
# w̅̅ᴸ = solution.w̅̅ᴸ
# w̅̅ᵁ = solution.w̅̅ᵁ
display(L"ALD = %$(intervalVectorLaTeXString(W))")

L"$ALD = \begin{pmatrix} \left[ 0.248, 0.413 \right] \\ \left[ 0.248, 0.372 \right] \\ \left[ 0.124, 0.228 \right] \\ \left[ 0.046, 0.19 \right] \\ \left[ 0.016, 0.115 \right] \end{pmatrix}$"

In [22]:
solution = MMR(A, GM)
W = solution.W
# w̅̅ᴸ = solution.w̅̅ᴸ
# w̅̅ᵁ = solution.w̅̅ᵁ
display(L"GM = %$(intervalVectorLaTeXString(W))")

L"$GM = \begin{pmatrix} \left[ 0.266, 0.401 \right] \\ \left[ 0.235, 0.385 \right] \\ \left[ 0.118, 0.251 \right] \\ \left[ 0.064, 0.147 \right] \\ \left[ 0.022, 0.112 \right] \end{pmatrix}$"